File: prices.ipynb\
Author: alexkobz\
Date: 08-10-2024\
Task: DSRFU-298

#### Шаг 0. Импорт и получение токена

In [ ]:
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt
from math import ceil
from src.utils.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = last_day_month
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)
REPORT_DATE

In [ ]:
from src.sources.rudata.RuDataMethod import Account, FintoolReferenceData, EndOfDay, AccruedInterestOnDate, FloaterData, FloatersOnPeriod, RUPriceHistory, ExchangeTree, MoexSecurities, CurrencyRate, HistoryStockBonds, HistoryStockShares, HistoryStockNdm, HistoryStockCcp
Acc = Account()

#### Шаг 1. RuData

##### Шаг 1.0. RuData stage

In [ ]:
FintoolReferenceDataDF = FintoolReferenceData().df
FintoolReferenceDataDF.head()

In [ ]:
EndOfDayDF = EndOfDay().df
EndOfDayDF.head()

In [ ]:
AccruedInterestOnDateDF = AccruedInterestOnDate().df
AccruedInterestOnDateDF.head()

In [ ]:
FloaterDataDF = FloaterData().df
FloaterDataDF.head()

In [ ]:
FloatersOnPeriodDF = FloatersOnPeriod().df
FloatersOnPeriodDF.head()

In [ ]:
RUPriceHistoryDF = RUPriceHistory().df
RUPriceHistoryDF.head()

In [ ]:
ExchangeTreeDF = ExchangeTree().df
ExchangeTreeDF.head()

In [ ]:
MoexSecuritiesDF = MoexSecurities().df
MoexSecuritiesDF.head()

In [ ]:
CurrencyRateDF = CurrencyRate().df
CurrencyRateDF.head()

##### Шаг 1.1. RuData MOEX stage

In [ ]:
HistoryStockBondsDF = HistoryStockBonds().df
HistoryStockBondsDF.head()

In [ ]:
HistoryStockSharesDF = HistoryStockShares().df
HistoryStockSharesDF.head()

In [ ]:
HistoryStockNdmDF = HistoryStockNdm().df
HistoryStockNdmDF.head()

In [ ]:
HistoryStockCcpDF = HistoryStockCcp().df
HistoryStockCcpDF.head()

##### Шаг 1.1. Discounts

In [ ]:
# Prices = pd.DataFrame(
#     index=FintoolReferenceDataDF['isincode'].dropna().unique(),
#     columns=
#         ["Name", "Type", "Principal", "Currency", "Clean price pct", "Clean price", "Interest", "Price",
#         "Source", "ISIN in DB", "FX Rate", "RUB Price", "CBR discount 6D", "Discount NCC", "NCC collateral"]
# )
# Prices.index.name = 'ISIN'
# Prices["Date"] = REPORT_DATE

##### Шаг 2. RuData transform

In [ ]:
Prices = FintoolReferenceDataDF[['isincode', 'nickname', 'fintooltype', 'fintoolid', 'faceftname']]\
.rename(columns={
    "isincode": "ISIN",
    "nickname": "Name",
    "fintoolid": "Fintool",
    "fintooltype": "Type",
    "faceftname": "Currency"
})
Prices = Prices[~(Prices["ISIN"].isna() | (Prices["ISIN"].isin(['None', ''])))]
InterestDF = AccruedInterestOnDateDF[['fintoolId', 'accruedInterest', 'currentFaceValue']].set_index('fintoolId')
InterestDF.update(FloatersOnPeriodDF.groupby('fintoolId').last())
Prices = Prices.merge(
    InterestDF, how="left", left_on="Fintool", right_index=True)
Prices = Prices.rename(columns={
    'accruedInterest': 'Interest',
    'currentFaceValue': 'Principal'})
Prices = Prices.merge(
    CurrencyRateDF[['currency', 'rate']], how='left', left_on='Currency', right_on='currency'
).drop(columns=['currency'])

###### Шаг 1.1.1. CBR

In [ ]:
from src.sources.cbr.CBR_Soap import CBR_Soap

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.6834.111 Safari/537.36', 
           'Accept-Encoding': 'gzip, deflate', 
           'Accept': '*/*', 
           'Connection': 'keep-alive'}
cbr_disc = CBR_Soap()\
.get_discounts(date=last_work_date_month_str)[['ISIN', 'от 1 месяца до 180 дней_Beg']]\
.rename(columns={'от 1 месяца до 180 дней_Beg': 'CBR discount 6D'})\
.groupby('ISIN')\
.last()\
.reset_index()

In [ ]:
Prices = Prices.merge(cbr_disc, how='left', left_on='ISIN', right_on='ISIN')
Prices['CBR discount 6D'] = Prices['CBR discount 6D'].fillna(1.0)

###### Шаг 1.1.2. nationalclearingcentre Рыночные риски

In [ ]:
currMarketRatesResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/fondMarketRates?action=xls&xls=1&lang=ru&date={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)].reset_index(drop=True)
currMarketRates = currMarketRates\
.merge(MoexSecuritiesDF[['secid', 'isin']], how='left', left_on='Инструмент', right_on='secid')\
.rename(columns={'isin': 'ISIN'})\
.groupby('ISIN')\
.last()\
.reset_index()
currMarketRates = currMarketRates[currMarketRates['ISIN'].str.match('([A-Z]{2})([A-Z0-9]{9})([0-9]{1})$')]
currMarketRates = currMarketRates[
    (currMarketRates['Диапазон']==1) & (currMarketRates['Ставка риска падения цены,%'] != 100)
]
currMarketRates=currMarketRates[['ISIN', 'Ставка риска падения цены,%']]\
.rename(columns={'Ставка риска падения цены,%': 'Risk rate'})
# Формула из п.12.1 Методики НКЦ определения риск-параметров
currMarketRates['Discount NCC'] = currMarketRates['Risk rate'].apply(lambda x: min(30, ceil(x/np.sqrt(2)))/100)

In [ ]:
Prices = Prices.merge(currMarketRates[['ISIN', 'Discount NCC']], how='left', left_on='ISIN', right_on='ISIN')
Prices['Discount NCC'] = Prices['Discount NCC'].fillna(1.0)

###### Шаг 1.1.3. nationalclearingcentre Параметры ценных бумаг

In [ ]:
securInfoResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/securInfo?action=xls&lang=ru&settleDate={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\
.rename(columns={
    'ISIN ЦБ': 'ISIN',
    'Принимается в обеспеч.': 'NCC collateral'
})\
.dropna()\
.groupby('ISIN')\
.last()\
.reset_index()
securInfo = securInfo[securInfo['NCC collateral']=='Да']

In [ ]:
Prices = Prices.merge(securInfo, how='left', left_on='ISIN', right_on='ISIN')
Prices['NCC collateral'] = Prices['NCC collateral'].fillna('Нет')

##### Шаг 1.2. RuData transform

In [ ]:
RUPriceHistoryLast = RUPriceHistoryDF.groupby('isincode').last().rename(columns={
    'vp_pct': 'Clean price pct',
    'vp_pc': 'Clean price',
    'acc_int': 'InterestHistory'
}).reset_index()
ru_price = RUPriceHistoryLast[['isincode', 'Clean price pct', 'Clean price', 'InterestHistory']].merge(
    Prices[['ISIN', 'Type', 'Interest']], left_on='isincode', right_on='ISIN')
ccnrd_bonds_idx = ru_price["Type"]=='Облигация'
ru_price.loc[ccnrd_bonds_idx, "InterestHistory"] = ru_price.loc[ccnrd_bonds_idx, "InterestHistory"].combine_first(ru_price.loc[ccnrd_bonds_idx, "Interest"])
ru_price.loc[ccnrd_bonds_idx, "Price"] = ru_price.loc[ccnrd_bonds_idx, "Clean price"] + ru_price.loc[ccnrd_bonds_idx, "InterestHistory"].fillna(0)
ru_price.loc[~ccnrd_bonds_idx, "Price"] = ru_price.loc[~ccnrd_bonds_idx, "Clean price"]
ru_price["Source"] = 'RU Data (ЦЦ НРД)'

In [ ]:
exch_prices = EndOfDayDF.merge(
    ExchangeTreeDF[['id', 'fullname_rus']], how='left', left_on='id_trade_site', right_on='id')[['fintoolId', 'last', 'fullname_rus']]
exch_prices["Source"] = exch_prices['fullname_rus'].apply(lambda x: 'RU Data (' + str(x) + ')')
exch_prices = exch_prices\
.rename(columns={
    'fintoolId': 'Fintool',
    'last': 'PriceEX'
})
exch_prices = exch_prices.merge(Prices[["Fintool", "ISIN", "Type", "Principal", "Interest"]], how='inner')
exch_prices[['PriceEX', 'Principal', 'Interest']] = exch_prices[['PriceEX', 'Principal', 'Interest']].fillna(0)

exch_bonds = exch_prices['Type']=='Облигация'
exch_prices.loc[exch_bonds, "Clean price pct"] = exch_prices.loc[exch_bonds, "PriceEX"]
exch_prices.loc[exch_bonds, "Clean price"] = exch_prices.loc[exch_bonds, "Clean price pct"]/100 * exch_prices.loc[exch_bonds, "Principal"]
exch_prices.loc[exch_bonds, "Price"] = exch_prices.loc[exch_bonds, "Clean price"] + exch_prices.loc[exch_bonds, "Interest"]

exch_prices.loc[~exch_bonds, 'Clean price'] = exch_prices.loc[~exch_bonds, 'PriceEX']
exch_prices.loc[~exch_bonds, 'Price'] = exch_prices.loc[~exch_bonds, 'PriceEX']

##### Шаг 3.1. Tranform

In [ ]:
TransformHistoryStockBonds = HistoryStockBondsDF.groupby("isin")['close'].last().rename("Price").reset_index()
TransformHistoryStockShares = HistoryStockSharesDF.groupby("isin")['close'].last().rename("Price").reset_index()
TransformHistoryStockNdm = HistoryStockNdmDF.groupby("isin")['close'].last().rename("Price").reset_index()
TransformHistoryStockCcp = HistoryStockCcpDF.groupby("isin")['close'].last().rename("Price").reset_index()
HistoryStock = pd.concat(
    [TransformHistoryStockBonds, TransformHistoryStockShares, TransformHistoryStockNdm, TransformHistoryStockCcp]
).rename(columns={'isin': 'ISIN'})
HistoryStock['Source'] = 'RU Data (MOEX)'

#### Шаг 4. Finish

In [ ]:
source_prices = pd.concat([
    ru_price[["ISIN", "Clean price pct", "Clean price", "Price", "Source"]],
    exch_prices[["ISIN", "Clean price pct", "Clean price", "Price", "Source"]],
    HistoryStock
])

In [ ]:
result = Prices.merge(source_prices, how='inner', left_on='ISIN', right_on='ISIN')
result['isin_in_db'] = 1
result['export_timestamp'] = dt.now().replace(microsecond=0)
result['Date'] = pd.to_datetime(last_day_month)
result['Source'] = result['Source'].fillna(0)
result = result.sort_values('ISIN')

##### Шаг 4.2. Write excel

In [ ]:
result.to_csv('../data/Output/prices.csv', index=False)
result.to_excel('../data/Output/prices.xlsx', index=False)

In [ ]:
from src.utils.send_email import main
main("prices.xlsx", "prices.csv")